# Pipeline 0

In [1]:
from pipeline import T5Converter
import time
from pprint import pprint

main_virtuoso_noreranking


In [8]:
from pathlib import Path
import pandas as pd
import json
import os

data_path = Path("..") / "t5-for-sparql" / "data" / "lcquad2" / "train.json"
data2_path = Path("..") / "t5-for-sparql" / "data" / "lcquad2" / "test_KG_4211.json"
output_links_dir = Path("..") / "t5-for-sparql" / "falcon_links" / "0ents_0rels"
# os.environ['ES_ENTITY_INDEX'] = "wikidataentityindex"
# os.environ['ES_PROPERTY_INDEX'] = "wikidatapropertyindet"

In [2]:
converter = T5Converter()

In [5]:
import re

def retrieve_gold_links(masked_wikisparql):
  fragments = masked_wikisparql.upper().strip().split(' ')
  ents = []
  rels = []
  result_dict = {
    'ents': ents,
    'rels': rels,
  }
  for fragment in fragments:
    id_match = re.search('[QP][0-9]+',fragment)
    if id_match:
      id_raw = id_match.group(0)
      if id_raw[0] == 'Q':
        prefix = 'wdt:'
        uri = 'http://www.wikidata.org/entity/' + id_raw
        target_arr = ents
      elif id_raw[0] == 'P':
        prefix = 'wd:'
        uri =  'http://www.wikidata.org/prop/direct/' + id_raw
        target_arr = rels
      else:
        raise ValueError('ID does not start with P or Q')
      match_dict = {
        'id' : id_raw,
        'prefix' : prefix,
        'uri' : uri,
      }
      target_arr.append(match_dict)
  return result_dict

In [7]:
def pipe_batch(utterances, wikisparqls):
  batched = []
  link_batch_start = time.time()
  linked = list(map(lambda u: {'utterance':u, 'ents':[], 'rels':[], }, utterances))
  assert len(linked) == len(wikisparqls)

  # for i, link in enumerate(linked):
  #   gold_linked = retrieve_gold_links(wikisparqls[i])
  #   new_ents=link['ents']
  #   old_ents_set = set(x['id'] for x in link['ents'])
  #   for gold_ent in gold_linked['ents']:
  #       if gold_ent['id'] not in old_ents_set:
  #           new_ents.append(gold_ent)
  #   link['rels'] = []
  #   assert len(linked[i]['rels']) == 0

  link_batch_end = time.time()
  s = time.time()
  annotated = list(map(lambda l: {'utterance': l['utterance'], 'fragments':[]}, linked))
  e = time.time()
  assert len(linked) == len(annotated)
  annotator_time = e - s
  converter_time = 0
  for i, single_annotated in enumerate(annotated):
    # annotated = annotator.annotate(**single_linked)
    # try:
    #   annotator_time += e - s
    # except Exception as e:
    #   print("[Annotator Error]:", e)
    #   continue
    s = time.time()
    try:
      converted = converter.preprocess(**single_annotated, wikisparql=wikisparqls[i])
    except Exception as e:
      print("[Converter Error]:", e)
      continue
      
    e = time.time()
    converter_time += e - s
    batched.append([linked[i], annotated[i], converted])
  print("Link batch time:", link_batch_end - link_batch_start)
  print("Anno batch time:", annotator_time)
  print("Conv batch time:", converter_time)
  return batched

## Data

In [9]:
with open(data_path) as f:
  data_dict = json.load(f)

with open(data2_path) as f:
  data_dict.extend(json.load(f))

count = 0

for item in data_dict:
  paraphrased_question = item["paraphrased_question"]
  question = item["question"]
  if paraphrased_question == None:
    raise Exception
  if question == None:
    count += 1
    # print(len(paraphrased_question) > 2)
    # pprint(item)
  if item["sparql_wikidata"] == None:
    raise Exception
  

  if len(paraphrased_question) > 2000:
    paraphrased_question = question
  # item["new_question"] = paraphrased_question if len(paraphrased_question) > 2 and paraphrased_question != [] else question
  item["new_question"] = question

print("Missing question count:", count)

df = pd.DataFrame.from_dict(data_dict)
df = df.dropna(axis=0, subset=['new_question'])
df = df[~df.question.isin(['n/a', None, 'none', 'None'])]
n_entries = len(df)
print("Number of rows:", n_entries)

Missing question count: 71
Number of rows: 28247


In [10]:
assert(len(df[df.question.isna()]) == 0)

In [11]:
for data in df[0:5].iterrows():
    print(data[1].new_question)

What periodical literature does Delta Air Lines use as a moutpiece?
Who is the child of Ranavalona I's husband?
Is it true Jeff_Bridges occupation Lane Chandler and photographer ?
What is the pre-requisite of phase matter of Galinstan?
Which is the operating income for Qantas?


In [12]:
responses = []
batch_qns = []
batch_ans = []
truncated_data = df[:]
total_len = len(truncated_data)
last = 0
cutoff = 0

for i, data in enumerate(truncated_data.iterrows()):
    data = data[1]
    if i < cutoff:
        continue
    # print("Pipeline iter", i)
    question = data["new_question"]
    answer = data["sparql_wikidata"]
    # try
    #     linked, annotated, converted = pipe(question, answer)
    # except Exception as e:
    #     print(e)
    #     continue
    # responses.append([linked, annotated, converted])
    if (question and answer) and len(question) >= 2:
        batch_qns.append(question)
        batch_ans.append(answer)
    if ((i + 1) % 2000) == 0 or i == total_len - 1:
        print("\n[Pipeline1]:", f"Linking {last}-{i}")
        try:
            for linked, annotated, converted in pipe_batch(batch_qns, batch_ans):
                responses.append([linked, annotated, converted])
            batch_qns = []
            batch_ans = []
            last = i
        except Exception as e:
            raise e
            # print(e)
            # continue
        with open(output_links_dir / f"link_{i}.json", "w") as f:
            json.dump(responses, f, indent=2, separators=(',',':'))

[Pipeline1]: Linking 0-49
1 : ['What periodical literature does Delta Air Lines use as a moutpiece?']
1 : ['Who is the child of Ranavalona Is husband?']
1 : ['Is it true Jeff_Bridges occupation Lane Chandler and photographer ?']
1 : ['What is the pre-requisite of phase matter of Galinstan?']
1 : ['Which is the operating income for Qantas?']
1 : ['which cola starts with the letter p']
1 : ['Is the right ascension of malin 1 less than 15.1398?']
1 : ['What is the complete list of records released by Jerry Lee Lewis?']
1 : ['What is Mary Lou Rettons International Olympic Committee athlete ID.']
1 : ['Who won the prize at the sequel of the 1885 Wimbledon Championships- Gentlemens Singles?']
1 : ['Is it true that the carbon footprint of the iPhone X Max is 106?']
1 : ['Tell me the female beauty pageant that operates in all countries and contains the word model in its name?']
1 : ['Did Lindsey Vonn participate in the FIS Alpine World Ski Championships in 1999 and 2019?']
1 : ['When position 